In [107]:
from tensorflow import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
import pandas as pd
from sklearn.utils import shuffle
import tensorflow as tf
import xgboost as xgb
import joblib

# Current Best - 13 (0.156)

# AutoEncoder

## Encoder

In [108]:
original_dim = 27
intermediate_dim = 13
latent_dim = 1

inputs = keras.Input(shape=(original_dim,))
x = layers.Dense(intermediate_dim, activation='linear')(inputs)
#x = layers.Dense(intermediate_dim, activation='linear')(inputs)

h = layers.Dense(intermediate_dim, activation='relu')(x)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)


### Sampling Layer

In [109]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_sigma = inputs
        
        batch = tf.shape(z_mean)[0]
        
        dim = tf.shape(z_mean)[1]
        
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_sigma) * epsilon

z = Sampling()([z_mean, z_log_sigma])

## Decoder & VAE

In [110]:
# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
#x = layers.Dense(intermediate_dim, activation='linear')(x)

outputs = layers.Dense(original_dim, activation='linear')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

## Loss

In [111]:
#mse = keras.losses.MeanSquaredError()
#reconstruction_loss = mse(inputs, outputs)
#reconstruction_loss *= 27
reconstruction_loss_factor = 2500
reconstruction_loss = tf.keras.backend.mean(tf.keras.backend.square(inputs-outputs))

kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5

vae_loss = K.mean((reconstruction_loss_factor*reconstruction_loss) + kl_loss)
vae.add_loss(vae_loss)

## Data

In [112]:
dataset = pd.read_csv('datasets/Fs_B_O_DANCE_WALK_KIN_0.5sec.csv')

train_dataset = dataset.sample(frac=0.85, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)


train_dataset = np.asarray(train_dataset)
test_dataset = np.asarray(test_dataset)

x_train = train_dataset.reshape((len(train_dataset), np.prod(train_dataset.shape[1:])))
x_test = test_dataset.reshape((len(test_dataset), np.prod(test_dataset.shape[1:])))

print(len(x_train[0]))

No Training Samples: 34156
No Test Samples: 6028
27


## Train

In [113]:
optimizer=keras.optimizers.Adam(learning_rate=0.001)
vae.compile(optimizer=optimizer)

In [114]:
vae.fit(x_train, x_train,
        epochs=128,
        batch_size=16,
       )

Epoch 1/128
2135/2135 [==============================] - 3s 1ms/step - loss: 329.5782
Epoch 2/128
2135/2135 [==============================] - 3s 1ms/step - loss: 216.1588
Epoch 3/128
2135/2135 [==============================] - 3s 1ms/step - loss: 214.8717
Epoch 4/128
2135/2135 [==============================] - 3s 1ms/step - loss: 213.9515
Epoch 5/128
2135/2135 [==============================] - 3s 1ms/step - loss: 213.1076
Epoch 6/128
2135/2135 [==============================] - 3s 1ms/step - loss: 212.8296
Epoch 7/128
2135/2135 [==============================] - 3s 1ms/step - loss: 212.6207
Epoch 8/128
2135/2135 [==============================] - 3s 1ms/step - loss: 212.4688
Epoch 9/128
2135/2135 [==============================] - 3s 1ms/step - loss: 212.1942
Epoch 10/128
2135/2135 [==============================] - 3s 1ms/step - loss: 211.9754
Epoch 11/128
2135/2135 [==============================] - 3s 1ms/step - loss: 211.7210
Epoch 12/128
2135/2135 [============================

2135/2135 [==============================] - 3s 1ms/step - loss: 196.2146
Epoch 96/128
2135/2135 [==============================] - 3s 1ms/step - loss: 196.3900
Epoch 97/128
2135/2135 [==============================] - 3s 1ms/step - loss: 195.7313
Epoch 98/128
2135/2135 [==============================] - 3s 1ms/step - loss: 196.7302
Epoch 99/128
2135/2135 [==============================] - 3s 1ms/step - loss: 195.4985
Epoch 100/128
2135/2135 [==============================] - 3s 1ms/step - loss: 197.2702
Epoch 101/128
2135/2135 [==============================] - 3s 1ms/step - loss: 201.7824
Epoch 102/128
2135/2135 [==============================] - 3s 1ms/step - loss: 197.8907
Epoch 103/128
2135/2135 [==============================] - 3s 1ms/step - loss: 198.0412
Epoch 104/128
2135/2135 [==============================] - 3s 1ms/step - loss: 198.3322
Epoch 105/128
2135/2135 [==============================] - 3s 1ms/step - loss: 203.9488
Epoch 106/128
2135/2135 [=========================

## Test

### Sample Comparison & Emotion Classification

In [115]:
sample = np.asarray(x_test[0])
sample = sample.reshape(1,-1)
print(sample)

[[ 0.43072943  0.23804284  0.25137196  0.37880336  0.44684691  0.46263396
   0.33256589  0.34248499  0.286151    0.27887122  0.00795415 -0.14954139
  -0.03442787  0.98786149  0.15465787  0.14396525  0.13484668  0.41601316
   0.47129584  0.58002704  0.34174169  0.50147265  0.95584669  0.96214887
   1.15397984  0.74853783  1.06304759]]


In [116]:
mean, var, generated = encoder.predict(sample)
print(generated)

[[0.71370333]]


In [117]:
regen = decoder.predict(generated)
print(regen)

[[ 0.5439786   0.3368191   0.34847587  0.3368892   0.41552877  0.4204768
   0.3302599   0.33504462  0.28606403  0.2778035   0.01054765 -0.09400146
  -0.0681314   0.9541349   0.28898832  0.11457188  0.1409095   0.40015918
   0.42851758  0.26544657  0.28264362  0.29302424  1.0335169   1.0559022
   0.6597234   0.7024478   0.64625186]]


In [118]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [119]:
scaled_sample = scaler.transform(sample)

real_coordinates = (
    model_p.predict(scaled_sample),
    model_a.predict(scaled_sample),
    model_d.predict(scaled_sample)
)

scaled_regen = scaler.transform(regen)

generated_coordinates = (
    model_p.predict(scaled_regen),
    model_a.predict(scaled_regen),
    model_d.predict(scaled_regen)
)


print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % np.asarray(generated_coordinates))

Real: [[-0.18105087]
 [-0.09036874]
 [-0.06514029]]
Predicted: [[-0.2257404 ]
 [-0.2718483 ]
 [ 0.15213144]]


/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


### MAE & MSE

In [120]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [121]:
mean, var, generated = encoder.predict(x_test)
regen = decoder.predict(generated)

print(regen)

[[0.5500726  0.33897582 0.35079283 ... 0.7078881  0.7534719  0.6941365 ]
 [0.4941336  0.32496446 0.32859433 ... 0.1290052  0.14932716 0.12419671]
 [0.49897194 0.32108793 0.33140257 ... 0.30069375 0.32243818 0.28960386]
 ...
 [0.6021679  0.35517013 0.37400267 ... 0.80195427 0.8468406  0.841792  ]
 [0.57269037 0.34670514 0.35980994 ... 0.8476575  0.9007689  0.8397384 ]
 [0.49056947 0.32248306 0.3285337  ... 0.15112484 0.17137945 0.14672017]]


In [122]:
mae_errors = mean_absolute_error(x_test, regen, multioutput='raw_values')
mse_errors = mean_squared_error(x_test, regen, multioutput='raw_values')

features = ["max_hand_distance",
          "avg_l_hand_hip_distance",
          "avg_r_hand_hip_distance",
          "max_stride_length",
          "avg_l_hand_chest_distance",
          "avg_r_hand_chest_distance",
          "avg_l_elbow_hip_distance",
          "avg_r_elbow_hip_distance",
          "avg_chest_pelvis_distance",
          "avg_neck_chest_distance",
          "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
          "avg_total_body_volume",
          "avg_triangle_area_hands_neck",
          "avg_triangle_area_feet_hips",
          
          "l_hand_speed",
          "r_hand_speed",
          "l_foot_speed",
          "r_foot_speed",
          "neck_speed",
          
          "l_hand_acceleration_magnitude",
          "r_hand_acceleration_magnitude",
          "l_foot_acceleration_magnitude",
          "r_foot_acceleration_magnitude",
          "neck_acceleration_magnitude",
         ]

print("Overall MAE: " + str(mean_absolute_error(x_test, regen)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print("Example [Regen-Real]: " + str(regen[i][i]) + " - " + str(x_test[i][i]))
    print()

Overall MAE: 0.15670356043772807

==max_hand_distance==
MSE: 0.06012
MAE: 0.19185
Example [Regen-Real]: 0.5500726 - 0.4307294347930162

==avg_l_hand_hip_distance==
MSE: 0.01924
MAE: 0.10951
Example [Regen-Real]: 0.32496446 - 0.4309957160177905

==avg_r_hand_hip_distance==
MSE: 0.02399
MAE: 0.12261
Example [Regen-Real]: 0.33140257 - 0.574602981247248

==max_stride_length==
MSE: 0.01940
MAE: 0.10684
Example [Regen-Real]: 0.36242947 - 0.5436291098827964

==avg_l_hand_chest_distance==
MSE: 0.01152
MAE: 0.08035
Example [Regen-Real]: 0.42536807 - 0.4366265481499257

==avg_r_hand_chest_distance==
MSE: 0.01141
MAE: 0.08069
Example [Regen-Real]: 0.4126603 - 0.4853309428154969

==avg_l_elbow_hip_distance==
MSE: 0.00531
MAE: 0.05266
Example [Regen-Real]: 0.34532416 - 0.3809397134466188

==avg_r_elbow_hip_distance==
MSE: 0.00556
MAE: 0.05451
Example [Regen-Real]: 0.33311942 - 0.3194540220825596

==avg_chest_pelvis_distance==
MSE: 0.00000
MAE: 0.00007
Example [Regen-Real]: 0.28606498 - 0.2861509999

# PAD to Latent Space

In [123]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

import xgboost as xgb

xgb.set_config(verbosity=0)

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize

from sklearn.utils import shuffle
import math
import joblib

from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import time


## Create Dataset

In [124]:
reg_train_X = train_emotions
reg_test_X = test_emotions

reg_train_X

,EMOTION_P,EMOTION_A,EMOTION_D
29246,0.10,-0.70,-0.2
21394,0.60,0.50,0.2
24952,-0.60,-0.30,-0.3
28782,-0.60,-0.30,-0.3
897,-0.40,0.25,-0.1
...,...,...,...
33700,-0.40,0.25,-0.1
4309,-0.50,0.60,0.9
20408,-0.40,0.25,-0.1
9671,-0.35,0.70,-0.8


In [125]:
reg_test_X

,EMOTION_P,EMOTION_A,EMOTION_D
21736,-0.4,0.25,-0.1
25552,0.6,0.50,0.2
9752,-0.6,-0.30,-0.3
10058,-0.6,-0.30,-0.3
37982,0.6,-0.55,0.1
...,...,...,...
33937,0.6,0.50,0.2
11950,0.5,0.70,0.4
20652,-0.6,-0.30,-0.3
28398,-0.4,0.25,-0.1


In [126]:
mean, var, generated = encoder.predict(x_train)
reg_train_Y = generated

mean, var, generated = encoder.predict(x_test)
reg_test_Y = generated

print(len(reg_train_Y))
print(len(reg_test_Y))

34156
6028


## Train Regressor

In [127]:
model = xgb.XGBRegressor(
                    n_estimators=1500, learning_rate=0.05, max_depth=10, min_child_weight=5, 
                    reg_alpha=0.1, reg_lambda=1, gamma=0.0,
                    subsample=0.75, colsample_bytree=0.75, objective="reg:squarederror",
                    tree_method='gpu_hist'
                )

In [128]:
model.fit(reg_train_X, reg_train_Y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.75,
             enable_categorical=False, gamma=0.0, gpu_id=0,
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=10,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=1500, n_jobs=12, num_parallel_tree=1,
             predictor='auto', random_state=0, reg_alpha=0.1, reg_lambda=1,
             scale_pos_weight=1, subsample=0.75, tree_method='gpu_hist',
             validate_parameters=1, verbosity=None)

## Test Model

In [130]:
score = model.score(reg_train_X, reg_train_Y)
print("Training score: ", score)

pred_y = model.predict(reg_train_X)
mse = mean_squared_error(reg_train_Y, pred_y)
mae = mean_absolute_error(reg_train_Y, pred_y)

print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

Training score:  0.21450224006226748
MSE: 0.50
MAE: 0.56


In [131]:
pred_y = model.predict(reg_test_X)
mse = mean_squared_error(reg_test_Y, pred_y)
mae = mean_absolute_error(reg_test_Y, pred_y)

print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

MSE: 0.50
MAE: 0.57


In [146]:
index = 0

real_value = np.asarray([reg_test_Y[index]])

row=reg_test_X.iloc[index]
row = np.asarray([row])

predicted = model.predict(row)

print('Real: %s' % real_value[0])
print('Predicted: %s' % predicted)

Real: [0.6967958]
Predicted: [0.647487]


In [147]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [148]:
regen = decoder.predict(predicted)
print(regen)

[[ 0.5518506   0.3396051   0.35146886  0.34499368  0.41565     0.4207828
   0.33057868  0.33588266  0.28606454  0.27774927  0.00991045 -0.09116153
  -0.07128949  0.95351416  0.29623687  0.11524868  0.14204371  0.43524724
   0.4640965   0.2898261   0.30749616  0.31924236  1.1339846   1.1546081
   0.72193986  0.7683582   0.70810694]]


In [149]:
scaled_regen = scaler.transform(regen)
generated_coordinates = (
    model_p.predict(scaled_regen),
    model_a.predict(scaled_regen),
    model_d.predict(scaled_regen)
)


print('Real: %s' % np.asarray(reg_test_X.iloc[index]))
print('Predicted: %s' % [generated_coordinates[0][0], generated_coordinates[1][0], generated_coordinates[2][0]])

Real: [-0.4   0.25 -0.1 ]
Predicted: [-0.14019483, -0.23435989, 0.06783479]


/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


## RandomSearchCV

In [939]:
# A parameter grid for XGBoost
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'gamma': [0, 0.001, 0.01],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [1, 1.25],
        'alpha': [0.0, 0.25]
        }

n_iter = 300

In [940]:
model = xgb.XGBRegressor(
                    n_estimators=1500,
                    objective="reg:squarederror",
                    tree_method='gpu_hist'
                )


In [941]:
# Pleasure
# run randomized search
kfold = KFold(n_splits=5, shuffle=True)

random_search = RandomizedSearchCV(model, param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

start = time.time()
random_search.fit(reg_train_X, reg_train_Y)


print("GridSearchCV took %.2f seconds"
      " parameter settings." % ((time.time() - start)))

GridSearchCV took 2863.87 seconds parameter settings.


### Results

In [942]:
best_regressor = random_search.best_estimator_

print(best_regressor.get_params())

{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'enable_categorical': False, 'gamma': 0.001, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.00999999978, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 5, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 1500, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0.25, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.75, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 1, 'eta': 0.01, 'alpha': 0.25}


In [943]:
pred_y = best_regressor.predict(reg_test_X)
mse = mean_squared_error(reg_test_Y, pred_y)
mae = mean_absolute_error(reg_test_Y, pred_y)

print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

MSE: 0.98
MAE: 0.78
